# Testing file 
### where we evaluate Zhang's models using the test set

## Preliminaries

In [1]:
import pandas as pd
from tensorflow.keras.optimizers import Adam
from tensorflow.data import Dataset


from util.load_data import load_data
from util.evaluation import *
from models.zhang.models import FairLogisticRegression
from models.zhang.learning import train_loop as zhang_train

In [2]:
batch_size = 64
epochs = 100
lr = 0.001
opt = Adam(learning_rate=lr)

In [3]:
test_loop = 1

## Load data

In [4]:
data_name = 'adult-race'

In [5]:
x_train, y_train, a_train = load_data(data_name, 'train')
raw_data = (x_train, y_train, a_train)

In [6]:
xdim = x_train.shape[1]
ydim = y_train.shape[1]
adim = a_train.shape[1]
zdim = 8

In [7]:
train_data = Dataset.from_tensor_slices((x_train, y_train, a_train))
train_data = train_data.batch(batch_size, drop_remainder=True)
train_data

<BatchDataset shapes: ((64, 112), (64, 1), (64, 5)), types: (tf.float64, tf.float64, tf.float64)>

In [8]:
x_valid, y_valid, a_valid = load_data(data_name, 'valid')

valid_data = Dataset.from_tensor_slices((x_valid, y_valid, a_valid))
valid_data = valid_data.batch(batch_size, drop_remainder=True)

In [9]:
x_test, y_test, a_test = load_data(data_name, 'test')

test_data = Dataset.from_tensor_slices((x_test, y_test, a_test))
test_data = test_data.batch(batch_size, drop_remainder=True)

## Result file

In [10]:
header = "model_name", "clas_acc", "dp", "deqodds", "deqopp", "trade_dp", "trade_deqodds", "trade_deqopp"
results = []

## Testing loop
#### Each model is evalueted 5 times
#### In the end of each iteration we save the result

### Zhang for DP

In [11]:
fairdef = 'DemPar'

for i in range(test_loop):
    opt = Adam(learning_rate=lr)
    model = FairLogisticRegression(xdim, ydim, adim, batch_size, fairdef)
    zhang_train(model, raw_data, train_data, epochs, opt)

    Y, A, Y_hat, A_hat = fair_evaluation(model, valid_data)
    
    clas_acc, dp, deqodds, deqopp, confusion_matrix  = compute_metrics(Y, A, Y_hat, adim=adim)

    fair_metrics = (dp, deqodds, deqopp)
    tradeoff = []
    for fair_metric in fair_metrics:
        tradeoff.append(compute_tradeoff(clas_acc, fair_metric))

    result = ['Zhang4DP', clas_acc, dp, deqodds, deqopp, tradeoff[0], tradeoff[1], tradeoff[2]]

    results.append(result)

> Epoch | Class Loss | Adv Loss | Class Acc | Adv Acc
> 1 | 0.5199112296104431 | 1.3905614614486694 | 0.6526856763925729 | -0.0019756742118208098
> 2 | 0.4611121714115143 | 1.2602486610412598 | 0.8189655172413793 | -0.0019756742118208098
> 3 | 0.4426349401473999 | 1.1552791595458984 | 0.8263428381962865 | 0.03217551942213145
> 4 | 0.43559280037879944 | 1.079308032989502 | 0.8294098143236074 | 0.14266955030375514
> 5 | 0.43303149938583374 | 1.0243663787841797 | 0.8323110079575596 | 0.21482638054248193
> 6 | 0.43252211809158325 | 0.9834703207015991 | 0.834507625994695 | 0.2660117253700681
> 7 | 0.4330763518810272 | 0.9521521329879761 | 0.8359167771883289 | 0.30107472271754826
> 8 | 0.4342109262943268 | 0.9277064800262451 | 0.8367042440318302 | 0.3239112610209842
> 9 | 0.4356565475463867 | 0.9081811904907227 | 0.8379476127320955 | 0.3414427597737755
> 10 | 0.4372462034225464 | 0.8922717571258545 | 0.8389008620689655 | 0.3523844044942122
> 11 | 0.438873827457428 | 0.8793830871582031 | 0.83

### Zhang for Eq Odds

In [12]:
fairdef = 'EqOdds'

for i in range(test_loop):
    opt = Adam(learning_rate=lr)
    model = FairLogisticRegression(xdim, ydim, adim, batch_size, fairdef)
    zhang_train(model, raw_data, train_data, epochs, opt)

    Y, A, Y_hat, A_hat = fair_evaluation(model, valid_data)
    
    clas_acc, dp, deqodds, deqopp, confusion_matrix  = compute_metrics(Y, A, Y_hat, adim=adim)

    fair_metrics = (dp, deqodds, deqopp)
    tradeoff = []
    for fair_metric in fair_metrics:
        tradeoff.append(compute_tradeoff(clas_acc, fair_metric))

    result = ['Zhang4EqOdds', clas_acc, dp, deqodds, deqopp, tradeoff[0], tradeoff[1], tradeoff[2]]

    results.append(result)

> Epoch | Class Loss | Adv Loss | Class Acc | Adv Acc
> 1 | 0.6225697994232178 | 0.9331574440002441 | 0.5320374668435013 | 0.016757749277969886
> 2 | 0.5413908362388611 | 0.7949942946434021 | 0.6870855437665783 | 0.38334428552607325
> 3 | 0.470356822013855 | 0.8387948274612427 | 0.7843998673740054 | 0.40821165945232707
> 4 | 0.4523882269859314 | 0.8009197115898132 | 0.8260527188328912 | 0.4198578797695808
> 5 | 0.4458237290382385 | 0.7848763465881348 | 0.8311090848806366 | 0.4342395110693155
> 6 | 0.44354134798049927 | 0.7742348909378052 | 0.8335958222811671 | 0.4395859961642511
> 7 | 0.44286924600601196 | 0.7669211626052856 | 0.8355437665782494 | 0.44397923255472665
> 8 | 0.44291868805885315 | 0.7617388963699341 | 0.8370772546419099 | 0.44733632851975036
> 9 | 0.44331079721450806 | 0.7577712535858154 | 0.8377403846153846 | 0.4500302939579405
> 10 | 0.4438524842262268 | 0.7547512650489807 | 0.838776525198939 | 0.45239269448844444
> 11 | 0.444433331489563 | 0.7521641254425049 | 0.838983

### Zhang for Eq Opp

In [13]:
fairdef = 'EqOpp'

for i in range(test_loop):
    opt = Adam(learning_rate=lr)
    model = FairLogisticRegression(xdim, ydim, adim, batch_size, fairdef)
    zhang_train(model, raw_data, train_data, epochs, opt)

    Y, A, Y_hat, A_hat = fair_evaluation(model, valid_data)
    
    clas_acc, dp, deqodds, deqopp, confusion_matrix  = compute_metrics(Y, A, Y_hat, adim=adim)

    fair_metrics = (dp, deqodds, deqopp)
    tradeoff = []
    for fair_metric in fair_metrics:
        tradeoff.append(compute_tradeoff(clas_acc, fair_metric))

    result = ['Zhang4EqOpp', clas_acc, dp, deqodds, deqopp, tradeoff[0], tradeoff[1], tradeoff[2]]

    results.append(result)

> Epoch | Class Loss | Adv Loss | Class Acc | Adv Acc
> 1 | 0.5086170434951782 | 0.4394846558570862 | 0.6637516578249337 | -0.0019756742118208098
> 2 | 0.461321622133255 | 0.3287355303764343 | 0.8246850132625995 | 0.07602498860194765
> 3 | 0.4544013738632202 | 0.28555962443351746 | 0.8296170424403183 | 0.13578957682895407
> 4 | 0.45495688915252686 | 0.25399768352508545 | 0.8314820954907162 | 0.17126703040985594
> 5 | 0.45406633615493774 | 0.23822914063930511 | 0.833347148541114 | 0.1967560887652936
> 6 | 0.4538351595401764 | 0.22981365025043488 | 0.8345905172413793 | 0.21217386064858273
> 7 | 0.4541389048099518 | 0.22525933384895325 | 0.8356266578249337 | 0.22191358213399387
> 8 | 0.4548230469226837 | 0.22293925285339355 | 0.8366627984084881 | 0.22746729566184531
> 9 | 0.45577552914619446 | 0.22195583581924438 | 0.8379061671087533 | 0.23152896674937848
> 10 | 0.4569042921066284 | 0.22180694341659546 | 0.8386521883289124 | 0.23409859539659333
> 11 | 0.4581281244754791 | 0.22215902805328

## Saving into DF then CSV

In [14]:
result_df = pd.DataFrame(results, columns=header)
result_df

,model_name,clas_acc,dp,deqodds,deqopp,trade_dp,trade_deqodds,trade_deqopp
0,Zhang4DP,0.851230,0.143808,0.683714,0.425707,0.246049,0.758331,0.567569
1,Zhang4EqOdds,0.850898,0.141695,0.682141,0.421282,0.242936,0.757231,0.563549
2,Zhang4EqOpp,0.850066,0.140729,0.680052,0.416002,0.241481,0.755614,0.558626


In [15]:
result_df.to_csv(f'{data_name}-result/zhang-{epochs}.csv')